In [73]:
%load_ext autoreload
%autoreload 2

import numpy as np
import nibabel as nb
import os
opj = os.path.join

import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from prfpy.stimulus import PRFStimulus2D
from prfpy.grid import Iso2DGaussianGridder, Norm_Iso2DGaussianGridder
from prfpy.fit import Iso2DGaussianFitter, Norm_Iso2DGaussianFitter
from prfpy.timecourse import sgfilter_predictions
from utils.utils import *

In [77]:
#create stimulus
dm_1R = create_dm_from_screenshots(screenshot_path='/Users/marcoaqil/PRFMapping/PRFMapping-Raw/sub-001/ses-1/rawdata/sub-001_ses-1_Logs/sub-001_ses-1_task-1R_run-1_Logs/Screenshots')
dm_1S = create_dm_from_screenshots(screenshot_path='/Users/marcoaqil/PRFMapping/PRFMapping-Raw/sub-001/ses-1/rawdata/sub-001_ses-1_Logs/sub-001_ses-1_task-1S_run-1_Logs/Screenshots')
dm_2R = create_dm_from_screenshots(screenshot_path='/Users/marcoaqil/PRFMapping/PRFMapping-Raw/sub-001/ses-1/rawdata/sub-001_ses-1_Logs/sub-001_ses-1_task-2R_run-1_Logs/Screenshots')
dm_4F = create_dm_from_screenshots(screenshot_path='/Users/marcoaqil/PRFMapping/PRFMapping-Raw/sub-001/ses-1/rawdata/sub-001_ses-1_Logs/sub-001_ses-1_task-4F_run-1_Logs/Screenshots')
dm_4R = create_dm_from_screenshots(screenshot_path='/Users/marcoaqil/PRFMapping/PRFMapping-Raw/sub-001/ses-1/rawdata/sub-001_ses-1_Logs/sub-001_ses-1_task-4R_run-1_Logs/Screenshots')

discard_volumes = 5
dm_full = np.concatenate((dm_1R[:,:,discard_volumes:],
                          dm_1S[:,:,discard_volumes:],
                          dm_2R[:,:,discard_volumes:],
                          dm_4F[:,:,discard_volumes:],
                          dm_4R[:,:,discard_volumes:]), axis=-1)

prf_stim = PRFStimulus2D(screen_size_cm=70, 
                         screen_distance_cm=210, 
                         design_matrix=dm_full,
                         TR=1.5)

In [ ]:
data_path = '/Users/marcoaqil/PRFMapping/PRFMapping-Deriv-noflairtse-manual-hires'
subj = 'sub-001'

#create a single brain mask in epi space
mask_dict = {}
for task_name in ['1R', '1S', '2R', '4F', '4R']:
    mask_ses_1 = nb.load(opj(data_path,'fmriprep/'+subj+'/ses-1/func/'+subj+'_ses-1_task-'+task_name+'_run-1_space-T1w_desc-brain_mask.nii.gz')).get_data().astype(bool)
    mask_ses_2 = nb.load(opj(data_path, 'fmriprep/'+subj+'/ses-2/func/'+subj+'_ses-2_task-'+task_name+'_run-1_space-T1w_desc-brain_mask.nii.gz')).get_data().astype(bool)
    
    mask_dict[task_name] = mask_ses_1 + mask_ses_2
    
final_mask = mask_dict['1R'] + mask_dict['1S'] + mask_dict['2R'] + mask_dict['4F'] + mask_dict['4R']

final_mask_2 = np.sum([mask_dict[key] for key in mask_dict])

final_mask==final_mask_2

In [ ]:
#preparing the data
tc_dict = {}
for task_name in ['1R', '1S', '2R', '4F', '4R']:
    timecoursefile_ses_1 = nb.load(opj(deriv_path, 'fmriprep/'+subj+'/ses-1/func/'+subj+'_ses-1_task-'+task_name+'_run-1_space-T1w_desc-preproc_bold.nii.gz')
    timecoursefile_ses_2 = nb.load(opj(deriv_path, 'fmriprep/'+subj+'/ses-2/func/'+subj+'_ses-2_task-'+task_name+'_run-1_space-T1w_desc-preproc_bold.nii.gz')
       
    timecourse_ses_1 = sgfilter_predictions(timecoursefile_ses_1.get_data()[:,:,:,discard_volumes:],
                                            window_length=121)
    timecourse_ses_2 = sgfilter_predictions(timecoursefile_ses_2.get_data()[:,:,:,discard_volumes:],
                                            window_length=121)
        
    tc_dict[task_name] = (timecourse_ses_1+timecourse_ses_2)/2.0
               


timecourse_full=np.concatenate((tc_dict['1R'],tc_dict['1S'],tc_dict['2R'],tc_dict['4F'],tc_dict['4R']), axis=-1)
                  
#brain mask                  
timecourse_brain = timecourse_full[final_mask]
#exclude timecourses with zero variance
timecourse_brain_nonzerovar = timecourse_brain[np.where(np.var(timecourse_brain, axis=-1)>0)]

In [ ]:
#create gaussian grid
grid_nr = 20
max_ecc_size = 16
sizes, eccs, polars = max_ecc_size * np.linspace(0.25,1,grid_nr)**2, \
                    max_ecc_size * np.linspace(0.1,1,grid_nr)**2, \
                        np.linspace(0, 2*np.pi, grid_nr)

gg = Iso2DGaussianGridder(stimulus=prf_stim,
                          filter_predictions=True,
                          window_length=121)

In [ ]:
%%time
gf = Iso2DGaussianFitter(data=timecourse_brain_nonzerovar, gridder=gg, n_jobs=10)

gf.grid_fit(ecc_grid=eccs,
                 polar_grid=polars,
                 size_grid=sizes)

In [ ]:
#refine Gaussian fits
%%time
gf.iterative_fit(rsq_threshold=0.05, verbose=True)

In [ ]:
%%time
#now refit normalization model, starting from results of iterated Gaussian fitting
gg_norm = Norm_Iso2DGaussianGridder(stimulus=prf_stim,
                                   filter_predictions=True,
                                   window_length=121)

gf_norm = Norm_Iso2DGaussianFitter(data= timecourse_brain_nonzerovar,
                                   gridder=gg_norm,
                                   n_jobs=10)

#have to add a column since in current code syntax
#gridsearch_params always contains the CSS exponent parameter, even if it is not fit.
#whereas iterative_search_params does not contain it if it is not fit)
starting_params = np.insert(gf.iterative_search_params, -1, 1.0, axis=-1)

gf_norm.iterative_fit(rsq_threshold=0.15, gridsearch_params=starting_params, verbose=True)

current_result=np.copy(gf_norm.iterative_search_params)

In [ ]:
%%time
#if needed, the normalization model iterative fit can be run again 
#(this makes sense only if changing some values in iterative_search minimization for increased precision)
new_starting_params = np.insert(current_result, -1, 1.0, axis=-1)
gf_norm.iterative_fit(rsq_threshold=0.0, gridsearch_params=new_starting_params, verbose=True)
current_result=np.copy(gf_norm.iterative_search_params)

In [ ]:
#compare rsq between models (ideally should be AIC or BIC)
print(np.mean(gf_norm.iterative_search_params[gf_norm.rsq_mask,-1]))
print(np.mean(gf.iterative_search_params[gf_norm.rsq_mask,-1]))

In [ ]:
#save results for plotting

attempt = np.zeros((78,102,80,10))
ha = attempt.reshape((-1,10))
ha[np.ravel(np.var(timecourse_brain, axis=-1)>0) & np.ravel(final_mask)]=gf_norm.iterative_search_params

haha = ha.reshape((78,102,80,10))

for i in range(10):
    nb.Nifti1Image(haha[:,:,:,i], timecoursefile_ses_1.affine).to_filename('norm{}.nii.gz'.format(i))
